In [1]:
import numpy as np
import pandas as pd
import RF

In [9]:
RF_output = RF.RF(2, 75, 30, 1.5)
RF_output

{'CT': 0.01,
 'GA': 0.01,
 'IA': 0.01,
 'IL': 0.01,
 'IN': 0.02,
 'KY': 0.14,
 'MD': 0.03,
 'MO': 0.03,
 'NH': 0.03,
 'NJ': 0.19,
 'OR': 0.06,
 'TN': 0.03,
 'VA': 0.03,
 'VT': 0.01,
 'WA': 0.01,
 'WV': 0.38}

In [16]:
RF_keys = sorted(RF_output.keys())

In [15]:
average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
RF_average_plant_capacity = average_plant_capacity[average_plant_capacity['State'].isin(RF_keys)].reset_index(drop = True)
RF_average_plant_capacity

,Year,State,Coal,NG,Petro,Hydro,Solar,Wind
0,2015,CT,600469,510656,15599,21604,5813,0
1,2015,GA,5304375,1484390,10014,99452,7191,0
2,2015,IA,1490553,66614,1060,320048,0,244830
3,2015,IL,3207552,139280,1476,13828,9872,346686
4,2015,IN,3401334,560792,78706,76247,4107,410467
5,2015,KY,4538776,457671,538606,486211,0,0
6,2015,MD,1740700,267961,14521,811595,4569,86919
7,2015,MO,3266276,106893,1938,265836,4224,172179
8,2015,NH,468331,2017231,19507,37359,0,141165
9,2015,NJ,439774,754580,46867,4770,5023,10769
